# Задание 3. Регрессия (LinearRegression)
1. Сгенерировать подходящие исходные данные для проведения обучения
2. Провести кластеризацию с помощью pyspark.ml / pyspark.mllib
3. Провести расчет метрик
4. Сохранить полученные значения метрик в файл.

## Сгенерировать подходящие исходные данные для проведения обучения

In [1]:
import os
os.environ['SPARK_HOME'] = "D:/BigData/spark-3.5.3"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

# Подготовка синтетических данных для обучения
X, y = make_regression(n_samples=100, n_features=1, noise=10, random_state=42)
y = y.reshape(-1, 1)
data = pd.DataFrame(np.hstack([X, y]), columns=["feature", "label"])

## Провести кластеризацию с помощью pyspark.ml / pyspark.mllib

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


# Инициализация SparkSession
spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

df = spark.createDataFrame(data)


# Векторизация признаков
vectorAssembler = VectorAssembler(inputCols=["feature"], outputCol="features")
df = vectorAssembler.transform(df)


# Разделение данных на обучающую и тестовую выборки
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Создание модели линейной регрессии
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

# Обучение модели
lr_model = lr.fit(train_data)

# Прогнозирование на тестовых данных
predictions = lr_model.transform(test_data)

# Провести расчет метрик

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

# Оценка модели
evaluator_mse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
mse = evaluator_mse.evaluate(predictions)

evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)

evaluator_mae = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)

print(f"Mean Squared Error (MSE) = {mse}")
print(f"R Squared (R2) = {r2}")
print(f"Mean Absolute Error (MAE) = {mae}")

Mean Squared Error (MSE) = 111.1909459941159
R Squared (R2) = 0.9534325882512602
Mean Absolute Error\ (MAE) = 8.18739911608149


## Сохранить полученные значения метрик в файл.

In [5]:
with open("LinearRegressionMetrics.txt", "w") as file:
    print(f"Mean Squared Error (MSE) = {mse}", file=file)
    print(f"R Squared (R2) = {r2}", file=file)
    print(f"Mean Absolute Error (MAE) = {mae}", file=file)
    